In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KDTree
from sklearn.metrics import mean_squared_error 

C:\Users\Priyanshu\project_1\env\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.filterwarnings('ignore')

In [27]:
df=pd.read_parquet("BMTC.parquet.gzip",engine='pyarrow')

In [28]:
dfinput=pd.read_csv("Input.csv",index_col="Unnamed: 0")

In [29]:
def csd(lat1, lon1, lat2, lon2, r=6371):
    coordinates = lat1, lon1, lat2, lon2
#    phi1, lambda1, phi2, lambda2 = [
#        radians(c) for c in coordinates
#    ]    
    phi1=np.radians(lat1)
    lambda1=np.radians(lon1)
    phi2=np.radians(lat2)
    lambda2=np.radians(lon2)
    a = (np.square(np.sin((phi2-phi1)/2)) + np.cos(phi1) * np.cos(phi2) * 
         np.square(np.sin((lambda2-lambda1)/2)))
    d = 2*r*np.arcsin(np.sqrt(a))
    return d

In [30]:
def EstimatedTravelTime(df, test):
    tstamp = df['Timestamp'].astype("string").str.split(' ',expand=True)
    tstamp=tstamp.drop(0,axis=1)
    tstamp=tstamp[1].astype("string").str.split(':',expand=True)
    tstamp=tstamp.astype(int)
    df["Time"]=tstamp[0]*60 + tstamp[1] + tstamp[2]*(1/60)
    df["Time"]=df["Time"]-df["Time"][0]
    df=df.drop("Timestamp",axis=1)
    busid=df["BusID"].unique()
    test_s=test.drop(["Dest_Lat","Dest_Long"],axis=1)
    test_d=test.drop(["Source_Lat","Source_Long"],axis=1)

    time_report=test.copy()
    time_report=time_report.drop(["Source_Lat","Source_Long","Dest_Lat","Dest_Long"],axis=1)

    dist_report=test.copy()
    dist_report=dist_report.drop(["Source_Lat","Source_Long","Dest_Lat","Dest_Long"],axis=1)

    dist=csd(test["Source_Lat"],test["Source_Long"],test["Dest_Lat"],test["Dest_Long"])
    tt=[]
    print("train_data_prepaired")
    
    for i in busid:
        bus=df[(df["BusID"]==i)]
        X=bus.drop(["BusID","Speed","Time"],axis=1)
        kdt=KDTree(X)
        if (len(bus)<8):
            a,s=kdt.query(test_s,k=len(bus))
            a,d=kdt.query(test_d,k=len(bus))
        else:          
            a,s=kdt.query(test_s,k=8)
            a,d=kdt.query(test_d,k=8)
    
        time=np.array(bus["Time"])[d] - np.array(bus["Time"])[s]
        time=pd.DataFrame(time)
        time[(time<=0)]=10000
        time=time.T.min()
        time_report[i]=time
    
        lat_d=pd.DataFrame(np.array(bus["Latitude"])[d]).T.min()
        lat_s=pd.DataFrame(np.array(bus["Latitude"])[s]).T.max()
    
        long_d=pd.DataFrame(np.array(bus["Longitude"])[d]).T.min()
        long_s=pd.DataFrame(np.array(bus["Longitude"])[s]).T.max()    
    
        d_d=csd(lat_d,long_d,test["Dest_Lat"],test["Dest_Long"])
        d_s=csd(lat_s,long_s,test["Source_Lat"],test["Source_Long"])
        t_d=d_d + d_s
        dist_report[i]=t_d
    
    for k in range(len(test)):
        loc=loc=((5*dist_report.iloc[k] +1*dist_report.iloc[k]*time_report.iloc[k])==(5*dist_report.iloc[k] +1*dist_report.iloc[k]*time_report.iloc[k]).min())
        t=time_report.iloc[k][loc]    
        t=t+dist_report.iloc[k][loc]*6
        tt.append(np.array(t))
    test["ETT"] = np.array(tt)
    return(test)

In [31]:
%%time
pred=EstimatedTravelTime(df, dfinput)

train_data_prepaired
CPU times: total: 1min 55s
Wall time: 1min 25s


In [32]:
pred.head(15)

,Source_Lat,Source_Long,Dest_Lat,Dest_Long,ETT
0,12.941644,77.557335,12.942002,77.551605,3.197840
1,12.845487,77.662079,12.845881,77.667892,3.311639
2,12.973492,77.622871,12.957303,77.621246,15.814620
3,12.819298,77.688995,12.814241,77.692986,1.933516
4,12.973240,77.615402,13.016170,77.627800,36.830526
5,12.992674,77.711830,12.992256,77.717758,2.907606
6,13.085214,77.504883,13.089375,77.510239,3.290143
7,12.892545,77.637688,12.902433,77.631485,5.496022
8,12.914073,77.677109,12.920435,77.665199,3.416169
9,12.850122,77.691895,12.828327,77.681847,13.594658
